# ETL file to convert the geojson file into network files

In [2]:
import json 
import csv
import networkx as nx
import os, sys

In [15]:
nodefile = "vertex/vertex.csv"
edgefile = "edge/edge.csv"
jsonfile = "geojson/multigraph.json"
graphmlfile = "graphml/multigraph.graphml"
gmlfile = "gml/multigraph.gml"

if not os.path.exists("edge"):
    os.makedirs("edge")
if not os.path.exists("vertex"):
    os.makedirs("vertex")
if not os.path.exists("gml"):
    os.makedirs("gml")

In [16]:
with open(jsonfile, 'r') as jsfile:
    with open(nodefile, 'w+') as nodefiled:
        with open(edgefile, 'w+') as edgefiled:
            node = csv.writer(nodefiled)
            edge = csv.writer(edgefiled)
            # Header
            node.writerow(["# NodeID", "Lat", " Lon", "Layer"])
            edge.writerow(["# EdgeID", "Source NodeID", "Target NodeID", "Direction", "Layer"])
            for line in jsfile:
                jsentry = json.loads(line)
                if jsentry['properties']['type'] == "node":
                    node.writerow([
                            jsentry['_id']['$oid'], 
                            jsentry['geometry']['coordinates'][0], 
                            jsentry['geometry']['coordinates'][1], 
                            jsentry['properties']['layer']])
                if jsentry['properties']['type'] == "edge":
                    if jsentry['properties']['name'].startswith("54"):
                        jsentry['properties']['name'] = "None"
                    if 'direction' in jsentry['properties']:
                        if jsentry['properties']['direction'] == "Double sens":
                            direction = "TwoWay"
                        elif (jsentry['properties']['direction'] == "Sens inverse" 
                              or jsentry['properties']['direction'] == "Sens unique"):
                            direction = "OneWay"
                            
                        edge.writerow([
                            jsentry['properties']['mongo_org_id'],
                            jsentry['properties']['mongo_dest_id'],
                            jsentry['_id']['$oid'], 
                            direction,
                            jsentry['properties']['layer'], 
                            jsentry['properties']['name']])
                    else:
                        edge.writerow([
                            jsentry['properties']['mongo_org_id'],
                            jsentry['properties']['mongo_dest_id'],
                            jsentry['_id']['$oid'],
                            "TwoWay",
                            jsentry['properties']['layer'], 
                            jsentry['properties']['name']])

In [17]:
G = nx.DiGraph()

with open(nodefile, 'r') as node:
    reader = csv.reader(node)
    next(reader)
    for row in reader:
        lat = row[1]
        lon = row[2]
        ntype = row[3]
        G.add_node(row[0], lat=lat, lon=lon, type=ntype)

with open(edgefile, 'r') as node:
    reader = csv.reader(node)
    next(reader)
    for row in reader:
        G.add_edge(row[1], row[2], type=row[4], name=row[5])
        if row[3] == 'TwoWay':
            G.add_edge(row[2], row[1], type=row[4], name=row[5])
        
nx.write_graphml(G, graphmlfile)
nx.write_graphml(G, gmlfile)

In [18]:
print(G.number_of_nodes())
print(G.number_of_edges())

42678
51931
